# Introduction to microbiome analysis

This is a tutorial based on the [data](http://ocean-microbiome.embl.de/) from the paper ["Structure and function of the global ocean microbiome". Sunagawa, Coelho, Chaffron, et al., Science, 2015](http://www.sciencemag.org/content/348/6237/1261359.long).

## Hello World

You can run this *cell* by typing CTRL+ENTER or from the menu.

In [ ]:
print("Hello World")

You can run the cells out of order, but **variables** are preserved.

Another way of saying it is that you are manipulating an environment underneath

In [ ]:
name = "Luis"

In [ ]:
print("Hello {}".format(name))

## Some basic imports

These are general imports for data analysis in Python

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

A magic command to make plots appear *inline*:

In [ ]:
%matplotlib inline

Another two to make them look nicer

In [ ]:
from matplotlib import style
style.use('bmh')

### Load the data

Load the metadata (sheet index 8 is the right one). The original data is at [http://ocean-microbiome.embl.de/companion.html](http://ocean-microbiome.embl.de/companion.html). A copy is shipped with this tutorial for convenience

In [ ]:
meta = pd.read_excel('./data/OM.CompanionTables.xlsx', sheetname=8, index_col=0)

Evaluating a cell with just a single object results in this object being displayed:

In [ ]:
meta

Remove the non-mod_data lines at the bottom

In [ ]:
meta = meta.select(lambda ix: ix.startswith('TARA'))

Now, we need to do some ID manipulation

(Let's look at the tables in the supplement first, before we look at this obscure code)

In [ ]:
samples = pd.read_excel('./data/OM.CompanionTables.xlsx', sheetname=1)
samples.index = samples['PANGAEA sample identifier']
samples_label = samples['Sample label [TARA_station#_environmental-feature_size-fraction]']
meta.index = samples_label.reindex(meta.index)

In [ ]:
samples

### First look: just basic correlations between the metadata

Let us look at the relationship between temperature and alpha diversity

In [ ]:
fig,ax = plt.subplots()
ax.scatter(meta['Mean_Temperature [deg C]*'], meta['miTAG.SILVA.Shannon'])

We can also compute a Spearman correlation to quantify the relationship:

In [ ]:
from scipy import stats
stats.spearmanr(meta['Mean_Temperature [deg C]*'], meta['miTAG.SILVA.Shannon'])

Hmmm, we are mixing apples and oranges...

In [ ]:
meta_prok = meta.select(lambda ix: ('0.22-3' in ix or '0.22-1.6' in ix))
meta_prok_srf = meta_prok.select(lambda ix: '_SRF_' in ix)

Now, redo the correlation:

In [ ]:
fig,ax = plt.subplots()
ax.scatter(meta_prok_srf['Mean_Temperature [deg C]*'], meta_prok_srf['miTAG.SILVA.Shannon'], s=30)
print(stats.spearmanr(meta_prok_srf['Mean_Temperature [deg C]*'], meta_prok_srf['miTAG.SILVA.Shannon']))

## Shannon vs Richness?

We have looked at alpha diversity using the Shannon index, but there are other options. Would it make a big difference?

In [ ]:
fig,ax = plt.subplots()
ax.scatter(meta_prok_srf['miTAG.SILVA.Taxo.Richness'], meta_prok_srf['miTAG.SILVA.Shannon'], s=30)
print(stats.spearmanr(meta_prok_srf['miTAG.SILVA.Taxo.Richness'], meta_prok_srf['miTAG.SILVA.Shannon']))

# Analysis of function

Let's now look at the functional tables, in particular at KEGG modules

In [ ]:
mod_data = pd.read_table('./data/TARA243.KO-module.profile.release', index_col=0)
print(mod_data)

We need to normalize the data:

In [ ]:
mod_data = (mod_data /mod_data.sum())
mod_data = mod_data.T
mod_data = mod_data.reindex(meta.index)

In [ ]:
data_prok_srf = mod_data.reindex(meta_prok_srf.index)

### Let's look at photosynthesis
Module 161 is [Photosystem II](http://www.genome.jp/kegg-bin/show_module?M00161)

In [ ]:
fig,ax = plt.subplots()
ax.scatter(meta_prok_srf['Mean_Temperature [deg C]*'], data_prok_srf['M00161'])
stats.spearmanr(meta_prok_srf['Mean_Temperature [deg C]*'], data_prok_srf['M00161'])

We should take the logarithm for display purposes

In [ ]:
fig,ax = plt.subplots()
ax.scatter(meta_prok_srf['Mean_Temperature [deg C]*'], np.log10(data_prok_srf['M00161']))
stats.spearmanr(meta_prok_srf['Mean_Temperature [deg C]*'], np.log10(data_prok_srf['M00161']))

What about the taxonomy?

In [ ]:
miTags = pd.read_table('./data/miTAG.taxonomic.profiles.release.tsv', index_col=0)

Let's look at the data:

In [ ]:
miTags

We need to perform some "data wrangling" (a lot of data analysis is just data wrangling):

In [ ]:
miTags = miTags.T
miTags = miTags.select(lambda ix: ix.startswith('TARA'))
miTags = miTags.astype(float) # It simplifies the downstream code

In [ ]:
miTags

We need to normalize the data (some magicky code)

In [ ]:
miTags  = (miTags.T/ miTags.sum(1)).T

# PCA Plot

Do you remember what a PCA is?

In [ ]:
from sklearn import decomposition
pca = decomposition.PCA(2)

In [ ]:
pca_decomp = pca.fit_transform(miTags)

In [ ]:
pca_decomp

In [ ]:
fig,ax = plt.subplots()
ax.scatter(pca_decomp[:,0], pca_decomp[:,1])

In [ ]:
pca_decomp = pca.fit_transform(miTags.reindex(index=meta_prok.index))
depths = []
for ix in meta_prok.index:
    depths.append(ix.split('_')[2])
print(depths)

In [ ]:
depths = pd.Categorical(depths)
print(depths.codes)

In [ ]:
colors = ["rgbk"[dc] for dc in depths.codes]
fig,ax = plt.subplots()
ax.scatter(pca_decomp[:,0], pca_decomp[:,1], c=colors)

In [ ]:
pca_decomp = pca.fit_transform(np.log10(miTags.reindex(index=meta_prok.index) + 1e-6))

fig,ax = plt.subplots()
ax.scatter(pca_decomp[:,0], pca_decomp[:,1], c=colors, s=20)

What about the relationship between biodiversity and the temperature/latitude?

In [ ]:
shannon = stats.entropy(miTags.T)
fig,ax = plt.subplots()
ax.scatter(meta_prok['Mean_Temperature [deg C]*'],shannon,  c=colors, s=20)

In [ ]:
from matplotlib import cm
miTags_srf = miTags.reindex(index=meta_prok_srf.index)
shannon_srf = shannon = stats.entropy(miTags_srf.T)
fig,ax = plt.subplots()
ax.scatter(meta_prok_srf['Mean_Temperature [deg C]*'], shannon_srf, c=np.abs(meta_prok_srf['Mean_Lat*']),  cmap=cm.RdBu, s=120)

# Final Demonstration of interactive exploration

I am going to present this without comments

In [ ]:
#from IPython.html.widgets import interact
from ipywidgets import interact

@interact(col1=list(meta.columns)[1:], col2=list(meta.columns)[1:], just_srf=True)
def cross(col1, col2, just_srf):
    if just_srf:
        data = meta_prok_srf
    else:
        data = meta_prok

    fig,ax = plt.subplots()
    ax.scatter(data[col1], data[col2])
    ax.set_xlabel(col1)
    ax.set_ylabel(col2)
    print(stats.spearmanr(data[col1], data[col2]))

# More information on the tools we used

Note that none of these are actually specific to metaomics, they are just data analysis tools

- https://jupyter.org/
- http://www.numpy.org/
- http://pandas.pydata.org
- http://scikit-learn.org
